This will likely be the last public notebook I release for a while. I wanted to clean up my prior LGBM model and give myself a solid baseline before using other models.

I really need to find better ways of getting training and validation data. Grouping by user_id and sampling task_container_id?

There is a lot more work to do in generating better student/user features.

I generated the question2 file using bigquery to aggregate question percentage correct and count of questions from the train data with the original question data.

In [ ]:
#task_container_id should be monotonically increasing by user, but it isn't, should it be fixed?
#train['task_container_id'] = (
#    train
#    .groupby('user_id')['task_container_id']
#    .transform(lambda x: pd.factorize(x)[0])
#    .astype('int16')
#)

## Reading Data and Importing Libraries ##

In [ ]:
#!pip install ../input/python-datatable/datatable-0.11.0-cp37-cp37m-manylinux2010_x86_64.whl

In [ ]:
import riiideducation
#import datatable as dt
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score

In [ ]:
#train = dt.fread('/kaggle/input/riiid-test-answer-prediction/train.csv')
#del train[:, ('row_id', 'user_answer', 'prior_question_had_explanation')]
#train = train.sort(['user_id', 'timestamp'])
#train.head()

In [ ]:
#train = train[dt.f.content_type_id== 0, :]
#del(train[:,'content_type_id'])

In [ ]:
#train = train.to_pandas()
#train.head()

In [ ]:
train = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv',
                   usecols=[1, 2, 3],
                   dtype={'timestamp': 'int64',
                          'user_id': 'int32',
                          'content_id': 'int16',
                          'content_type_id': 'int8',
                          'task_container_id': 'int16',
                          'answered_correctly':'int8',
                          'prior_question_elapsed_time': 'float32'}
                   )

In [ ]:
train = train[train.content_type_id == 0]
train.drop('content_type_id', axis = 1, inplace = True)
train = train.sort_values(['user_id','timestamp'])

Affirmatives (True) for content_type_id are only for those with a different type of content (lectures). These are not real questions.

In [ ]:
#saving value to fillna
elapsed_mean = train.prior_question_elapsed_time.mean()

train.prior_question_elapsed_time.fillna(elapsed_mean, inplace = True)

## Saving Grouped Features ##

In [ ]:
group1 = train[['task_container_id', 'user_id']].groupby(['task_container_id']).agg(['count'])
group1.columns = ['avg_questions']
group2 = train[['task_container_id', 'user_id']].groupby(['task_container_id']).agg(['nunique'])
group2.columns = ['avg_questions']
group3 = group1 / group2
del(group1, group2)

In [ ]:
#supposed to be monotonically increasing by user, but not quite, close enough hopefully
group3['avg_questions_seen'] = group3.avg_questions.cumsum()

In [ ]:
import pickle as pkl

group3.to_pickle('./group3.pkl', compression='infer')

In [ ]:
results_u_final = train[['user_id','answered_correctly']].groupby(['user_id']).agg(['mean'])
results_u_final.columns = ['answered_correctly_user']



In [ ]:
results_u_final.to_pickle('./results_u_final.pkl', compression='infer')

## Data Exploration ##

In [ ]:
len(train)

In [ ]:
train.answered_correctly.mean()

## Timestamp Features ##

In [ ]:
train['time_since_last_action'] = train['timestamp'] - train.timestamp.shift(1)

train.time_since_last_action.replace(to_replace=0, method = 'ffill', inplace = True)

train.loc[train.time_since_last_action < 0, 'time_since_last_action'] = np.nan


In [ ]:
time_since_median = train.time_since_last_action.median()

train.time_since_last_action.fillna(time_since_median, inplace = True)

## Extracting Training Data ##

In [ ]:
X = train.sample(frac=.15, random_state = 100)
train = train[~train.index.isin(X.index)]
len(X) + len(train)

In [ ]:
X.answered_correctly.mean()

In [ ]:
train.answered_correctly.mean()

In [ ]:
X.task_container_id.value_counts()

## Creating Validation Set ##

In [ ]:
validation = train.sample(frac=.03, random_state = 100)

In [ ]:
train = train[~train.index.isin(validation.index)]
len(train) + len(validation) + len(X)

In [ ]:
validation.answered_correctly.mean()

In [ ]:
train.answered_correctly.mean()

Does it make sense to use last questions as validation? Why is the rate of correct answers so low?
I am convinced there is a better way to match the test data.

## Merging Data ##

In [ ]:
#clearing memory
del(train)

In [ ]:
pd.read_pickle("./.pkl")

In [ ]:
X = pd.merge(X, group3, left_on=['task_container_id'], right_index= True, how="left")
X = pd.merge(X, results_u_final, on=['user_id'], how="left")

In [ ]:
validation = pd.merge(validation, group3, left_on=['task_container_id'], right_index= True, how="left")
validation = pd.merge(validation, results_u_final, on=['user_id'], how="left")

In [ ]:
#reading in question df
question2 = pd.read_csv('/kaggle/input/question2/question2.csv')

In [ ]:
content_mean = question2.quest_pct.mean()

question2.quest_pct.mean()
#there are a lot of high percentage questions, should use median instead?

In [ ]:
#filling questions with no info with a new value
question2.quest_pct = question2.quest_pct.mask((question2['count'] < 4), .65)


#filling very hard new questions with a more reasonable value
question2.quest_pct = question2.quest_pct.mask((question2.quest_pct < .2) & (question2['count'] < 21), .2)

#filling very easy new questions with a more reasonable value
question2.quest_pct = question2.quest_pct.mask((question2.quest_pct > .95) & (question2['count'] < 21), .95)

In [ ]:
question2.to_pickle('./question2.pkl', compression='infer')

In [ ]:
X = pd.merge(X, question2, left_on = 'content_id', right_on = 'question_id', how = 'left')
validation = pd.merge(validation, question2, left_on = 'content_id', right_on = 'question_id', how = 'left')
X.part = X.part - 1
validation.part = validation.part - 1

In [ ]:
y = X['answered_correctly']
X = X.drop(['answered_correctly'], axis=1)
X.head()

y_val = validation['answered_correctly']
X_val = validation.drop(['answered_correctly'], axis=1)

In [ ]:
#try to concat this to test if it works later
#time_df = validation[['user_id', 'content_id', 'timestamp', 'time_since_last_action']].drop_duplicates('user_id', keep = 'last')
#not working on test data yet.

In [ ]:
X = X[['answered_correctly_user', 'quest_pct', 'avg_questions_seen', 'prior_question_elapsed_time', 'time_since_last_action', 'part', 'count']]
X_val = X_val[['answered_correctly_user', 'quest_pct', 'avg_questions_seen', 'prior_question_elapsed_time', 'time_since_last_action', 'part', 'count']]

In [ ]:
X['answered_correctly_user'].fillna(0.65,  inplace=True)
X['quest_pct'].fillna(content_mean, inplace=True)

X['part'].fillna(4, inplace = True)
X['avg_questions_seen'].fillna(1, inplace = True)
X['prior_question_elapsed_time'].fillna(elapsed_mean, inplace = True)


In [ ]:
X_val['answered_correctly_user'].fillna(0.65,  inplace=True)
X_val['quest_pct'].fillna(content_mean,  inplace=True)

X_val['part'].fillna(4, inplace = True)
X['avg_questions_seen'].fillna(1, inplace = True)
X_val['prior_question_elapsed_time'].fillna(elapsed_mean, inplace = True)

In [ ]:
print(len(X), len(X_val))

## Modeling ##

In [ ]:
import lightgbm as lgb

params = {
    'objective': 'binary',
    'boosting' : 'gbdt',
    'metric': 'auc'
    'max_bin': 1500,
    'learning_rate': 0.0175,
    'num_leaves': 80
}

lgb_train = lgb.Dataset(X, y, categorical_feature = ['part'])
lgb_eval = lgb.Dataset(X_val, y_val, categorical_feature = ['part'], reference=lgb_train)

In [ ]:
del(X, y)

In [ ]:
model = lgb.train(
    params, lgb_train,
    valid_sets=[lgb_train, lgb_eval],
    verbose_eval=50,
    num_boost_round=1000,
    early_stopping_rounds=8
)

In [ ]:
y_pred = model.predict(X_val)
y_true = np.array(y_val)
roc_auc_score(y_true, y_pred)

## Examining Feature Importance ##

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#displaying the most important features by split
lgb.plot_importance(model)
plt.show()

In [ ]:
#displaying the most important features by gain
lgb.plot_importance(model, importance_type = 'gain')
plt.show()

In [ ]:
# save model to file
model.save_model('./lgbm_model.txt')

In [ ]:
bst = lgb.Booster(model_file='./lgbm_model.txt')
#y_pred = bst.predict(X_test)

In [ ]:
X_val.iloc[0].shape

In [ ]:
X_val.iloc[0]

## Making Predictions for New Data ##

In [ ]:
env = riiideducation.make_env()
iter_test = env.iter_test()

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    test_df['task_container_id'] = test_df.task_container_id.mask(test_df.task_container_id > 9999, 9999)
    test_df = pd.merge(test_df, group3, left_on=['task_container_id'], right_index= True, how="left")
    test_df = pd.merge(test_df, question2, left_on = 'content_id', right_on = 'question_id', how = 'left')
    test_df = pd.merge(test_df, results_u_final, on=['user_id'],  how="left")
    test_df['answered_correctly_user'].fillna(0.65,  inplace=True)
    test_df['quest_pct'].fillna(content_mean,  inplace=True)
    test_df['part'] = test_df.part - 1

    test_df['part'].fillna(4, inplace = True)
    test_df['avg_questions_seen'].fillna(1, inplace = True)
    test_df['prior_question_elapsed_time'].fillna(elapsed_mean, inplace = True)
    test_df['prior_question_had_explanation'].fillna(False, inplace=True)
    
    test_df['answered_correctly'] =  model.predict(test_df[['answered_correctly_user', 'quest_pct', 'avg_questions_seen', 
                                                            'prior_question_elapsed_time', 'time_since_last_action', 'part', 'count']])
    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])

In [ ]:
time_since_median

In [ ]:
#students don't appear in every task container ID what can I do about this, can't always follow sequentially?

In [ ]:
#secret code for making calculation work
'''time_df = pd.concat([time_df, test_df[['user_id', 'content_id', 'timestamp']]], axis=0, ignore_index = True)

time_df = time_df.sort_values(['user_id','timestamp'])

time_df['prev_timestamp'] = time_df.timestamp.shift(1)

time_df['time_since_last_action'] = time_df['timestamp'] - time_df['prev_timestamp']

time_df.time_since_last_action.replace(to_replace=0, method = 'ffill', inplace = True)

time_df.loc[time_df.time_since_last_action < 0, 'time_since_last_action'] = np.nan

time_df.time_since_last_action.fillna(time_since_median, inplace = True)

test_df = pd.merge(test_df, time_df, on=['user_id', 'content_id', 'timestamp'], how = "left")

test_df.time_since_last_action.fillna(time_since_median, inplace = True)'''

#use map instead if you only have a single column
